# Analiza hribov in poti v Sloveniji

Najprej sem uvozil knjižnico Pandas, s katero sem opravil analizo podatkov. Uvozil in preberem csv datoteki s shranjenimi hribi (hribiPop.csv) in potmi (potiMin.csv). Podatke za analizo sem zbral s pletnih strani in shranil v format csv s pomočjo skripte izlušči imena hribov.py. 

In [23]:
import pandas as pd
pd.options.display.max_rows = 15

hribi = pd.read_csv('hribiPop.csv', delimiter=';', index_col='Ime')
poti = pd.read_csv('potiMin.csv', delimiter=';', index_col='Ime')


In [24]:
hribi.head(10)

,Gorovje,Višina,Število_poti,Vrsta_cilja
Ime,,,,
Ahac,"Goriško, Notranjsko in Snežniško hribovje",799,6,"'vrh', 'cerkev'"
Ajdovščina,"Goriško, Notranjsko in Snežniško hribovje",804,1,'vrh'
Ankaran,"Goriško, Notranjsko in Snežniško hribovje",18,3,''
Baba (pri Javornikih),"Goriško, Notranjsko in Snežniško hribovje",1085,2,'vrh'
Bela stena,"Goriško, Notranjsko in Snežniško hribovje",1122,1,'vrh'
Bele ovce,"Goriško, Notranjsko in Snežniško hribovje",1029,1,'vrh'
Bele stene,"Goriško, Notranjsko in Snežniško hribovje",863,2,''
Belovka,"Goriško, Notranjsko in Snežniško hribovje",848,2,'vrh'
Bezgarska planina,"Goriško, Notranjsko in Snežniško hribovje",895,4,'planina'


In [33]:
hribi = hribi.assign(row_number=range(len(hribi)))
hribi

,Gorovje,Višina,Število_poti,Vrsta_cilja,row_number
Ime,,,,,
Ahac,"Goriško, Notranjsko in Snežniško hribovje",799,6,"'vrh', 'cerkev'",0
Ajdovščina,"Goriško, Notranjsko in Snežniško hribovje",804,1,'vrh',1
Ankaran,"Goriško, Notranjsko in Snežniško hribovje",18,3,'',2
Baba (pri Javornikih),"Goriško, Notranjsko in Snežniško hribovje",1085,2,'vrh',3
Bela stena,"Goriško, Notranjsko in Snežniško hribovje",1122,1,'vrh',4
...,...,...,...,...,...
Zeleno okno,"Strojna, Košenjak, Kozjak in Slovenske gorice",325,3,'vrh',2102
Župank,"Strojna, Košenjak, Kozjak in Slovenske gorice",508,1,'',2103
Župank,"Strojna, Košenjak, Kozjak in Slovenske gorice",508,1,'',2104


In [25]:
poti.head(10)

,Višinska_razlika,Zahtevnost,Čas
Ime,,,
Ahac,189,lahka označena pot,40
Ahac,300,lahka označena pot,120
Ahac,379,lahka neoznačena steza,60
Ahac,377,lahka neoznačena steza,60
Ahac,175,lahka označena pot,60
Ahac,69,lahka neoznačena steza,20
Ajdovščina,225,lahka neoznačena steza,40
Ankaran,400,lahka označena pot,210
Ankaran,200,lahka označena pot,120


**1.** *V katerem gorovju imajo hribi v povprečju največ poti?*

In [28]:
hribi_po_gorovju = hribi.groupby('Gorovje')

In [30]:
hribi_po_gorovju.Število_poti.sum()

Gorovje
Goriško, Notranjsko in Snežniško hribovje         819
Julijske Alpe                                    1461
Kamniško Savinjske Alpe                          1162
Karavanke                                        1030
Pohorje, Dravinjske gorice in Haloze              377
Polhograjsko hribovje in Ljubljana                363
Posavsko hribovje in Dolenjska                    740
Prekmurje                                         150
Strojna, Košenjak, Kozjak in Slovenske gorice     260
Škofjeloško, Cerkljansko hribovje in Jelovica     611
Name: Število_poti, dtype: int64

In [31]:
hribi_po_gorovju.Število_poti.mean()

Gorovje
Goriško, Notranjsko in Snežniško hribovje        2.804795
Julijske Alpe                                    2.933735
Kamniško Savinjske Alpe                          4.179856
Karavanke                                        3.355049
Pohorje, Dravinjske gorice in Haloze             3.490741
Polhograjsko hribovje in Ljubljana               3.945652
Posavsko hribovje in Dolenjska                   3.109244
Prekmurje                                        2.542373
Strojna, Košenjak, Kozjak in Slovenske gorice    2.765957
Škofjeloško, Cerkljansko hribovje in Jelovica    4.333333
Name: Število_poti, dtype: float64

**2.** *Višji hribi imajo zahtevnejše poti.*

TODO: 1.povezati ime hriba z njegovimi potmi,
2. Zahtevnost poti pretvoriti v število oziroma ordinalno spremenljivko, 
3. Za vsak hrib izračunati povprečje zahtevnosti poti, 
4. Primerjati višino hribov in njihovo povprečno zahtevnost (npr. scatter plot)